# Introduction to the Low-salt-rejection Reverse Osmosis (LSRRO) Flowsheet
This tutorial shows how to build, initialize, simulate, and optimize an LSRRO flowsheet using WaterTAP.

![alt text](../docs/_static/flowsheets/lsrro.png)

## High-level Preview: Run an LSRRO optimization case.
#### 3-stage system
#### 70 g/L TDS feed, 50% Recovery


In [1]:
from watertap.flowsheets.lsrro.lsrro import (
    ACase,
    BCase,
    ABTradeoff,
    run_lsrro_case,
)
import logging
logging.getLogger("idaes").setLevel(logging.ERROR)
logging.getLogger("pyomo").setLevel(logging.ERROR)

In [2]:
# %%capture 
m, results = run_lsrro_case(
    number_of_stages=3,
    water_recovery=0.50,
    Cin=70,  # inlet NaCl conc kg/m3,
    Qin=1e-3,  # inlet feed flowrate m3/s
    A_case=ACase.optimize,
    B_case=BCase.optimize,
    AB_tradeoff=ABTradeoff.equality_constraint,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
    permeate_quality_limit=1000e-6,
    AB_gamma_factor=1,
    # B_max=3.5e-6,
    number_of_RO_finite_elements=10,
    skip_initialization=True
)


Feed Concentration = 70.0 ppt

***---Simulation results---***
Simulation failed. The current configuration is infeasible. Please adjust the decision variables.

***---Optimization results---***
----system metrics----
Feed: 1.05 kg/s, 66946 ppm
Product: 0.498 kg/s, 1000 ppm
Brine: 0.548 kg/s, 126890 ppm
Volumetric water recovery: 50.0%
Number of Stages: 3
Total Membrane Area: 265.86
Energy Consumption: 7.6 kWh/m3
Levelized cost of water: 1.71 $/m3
Primary Pump Capital Cost ($/m3):0.5047265838517884
Booster Pump Capital Cost ($/m3): 0.1318504520423668
ERD Capital Cost ($/m3):0.04116999386381928
Membrane Capital Cost ($/m3): 0.14321735175018716
Indirect Capital Cost ($/m3): 0.4104821907540809
Electricity cost ($/m3): 0.5323462487411282


--decision variables--
Stage 1 operating pressure 85.0 bar
Stage 1 membrane area      156.2 m2
Stage 1 water perm. coeff.  1.5 LMH/bar
Stage 1 salt perm. coeff.  0.1 LMH
Stage 2 operating pressure 65.0 bar
Stage 2 membrane area      97.3 m2
Stage 2 water 

## Overview of setting up and optimizing the LSRRO flowsheet--step by step.
* Step 0: Import Python libraries.
* Step 1: Build the LSRRO flowsheet.
* Step 2: Set specifications for LSRRO flowsheet.
* Step 3: Initialize the LSRRO flowsheet.
* Step 4: Optimize the LSRRO flowsheet.
* Step 5: Visualize results.

## Step 0: Import libraries.

In [3]:
# Import LSRRO flowsheet module
from watertap.flowsheets.lsrro import lsrro

## Step 1: Build the LSRRO flowsheet.

In [4]:
# Create a Pyomo concrete model, flowsheet, and NaCl property parameter block.
m = lsrro.build(
    number_of_stages=3,
    has_NaCl_solubility_limit=True,
    has_calculated_concentration_polarization=True,
    has_calculated_ro_pressure_drop=True,
)


## Step 2: Set specifications for the LSRRO flowsheet.

In [5]:
lsrro.set_operating_conditions(m)

Feed Concentration = 70.0 ppt


## Step 3: Initialize the LSRRO flowsheet model.

In [6]:
lsrro.initialize(m)

--------------------START FORWARD INITIALIZATION PASS--------------------
--------------------START BACKWARD INITIALIZATION PASS--------------------
--------------------START FORWARD INITIALIZATION PASS--------------------
INFO: Starting Sequential Decomposition
INFO: Starting first pass run of network
2025-09-02 06:47:25 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-09-02 06:47:26 [INFO] idaes.init.fs.PrimaryPumps[1].control_volume: Initialization Complete
2025-09-02 06:47:26 [INFO] idaes.init.fs.PrimaryPumps[1]: Initialization Complete: optimal - Optimal Solution Found
2025-09-02 06:47:27 [INFO] idaes.init.fs.Mixers[1]: Initialization Complete: optimal - Optimal Solution Found
2025-09-02 06:47:27 [INFO] idaes.init.fs.ROUnits[1].feed_side: Initialization Complete
2025-09-02 06:47:30 [INFO] idaes.init.fs.ROUnits[1]: Initialization Complete: optimal - Optimal Solution Found
2025-09-02 06:47:30 [INFO] idaes.init.fs.EnergyRecoveryDevices[1].control_volume: Initialization Comple

## Step 4: Simulate the LSRRO flowsheet.

In [7]:
lsrro.solve(m)


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 2039, 'Number of variables': 2039, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.2259519100189209}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
lsrro.display_state(m)

--------state---------
Feed                : 1.046 kg/s, 66946 ppm, 1.0 bar
Primary Pump 1 out  : 1.046 kg/s, 66946 ppm, 75.0 bar
Mixer 1 recycle     : 0.395 kg/s, 27937 ppm, 75.0 bar
Mixer 1 out         : 1.440 kg/s, 56253 ppm, 75.0 bar
RO 1 permeate       : 0.349 kg/s, 751 ppm, 1.0 bar
RO 1 retentate      : 1.092 kg/s, 73988 ppm, 66.9 bar
Primary Pump 2 out  : 1.092 kg/s, 73988 ppm, 75.0 bar
Mixer 2 recycle     : 0.198 kg/s, 39703 ppm, 75.0 bar
Mixer 2 out         : 1.290 kg/s, 68722 ppm, 75.0 bar
RO 2 permeate       : 0.395 kg/s, 27937 ppm, 1.0 bar
RO 2 retentate      : 0.895 kg/s, 86717 ppm, 71.6 bar
Booster Pump 2 out  : 0.395 kg/s, 27937 ppm, 75.0 bar
Primary Pump 3 out  : 0.895 kg/s, 86717 ppm, 75.0 bar
RO 3 permeate       : 0.198 kg/s, 39703 ppm, 1.0 bar
RO 3 retentate      : 0.697 kg/s, 100084 ppm, 73.4 bar
Booster Pump 3 out  : 0.198 kg/s, 39703 ppm, 75.0 bar
Disposal            : 0.697 kg/s, 100084 ppm, 1.0 bar
Product             : 0.349 kg/s, 751 ppm, 1.0 bar


## Step 5: Optimize the LSRRO flowsheet.

In [9]:
lsrro.optimize_set_up(m,
    water_recovery=0.5,
    A_case=ACase.optimize,
    B_case=BCase.optimize,
    AB_tradeoff=ABTradeoff.equality_constraint,
    permeate_quality_limit=500e-6,
    )
optimization_results = lsrro.solve(m, raise_on_failure=True)

In [10]:
lsrro.display_system(m)

----system metrics----
Feed: 1.05 kg/s, 66946 ppm
Product: 0.498 kg/s, 500 ppm
Brine: 0.548 kg/s, 127301 ppm
Volumetric water recovery: 50.0%
Number of Stages: 3
Total Membrane Area: 242.69
Energy Consumption: 9.0 kWh/m3
Levelized cost of water: 1.94 $/m3
Primary Pump Capital Cost ($/m3):0.5508102736918976
Booster Pump Capital Cost ($/m3): 0.18165719719739787
ERD Capital Cost ($/m3):0.04458632368629701
Membrane Capital Cost ($/m3): 0.1469527167064308
Indirect Capital Cost ($/m3): 0.46200325564101163
Electricity cost ($/m3): 0.6276017013441124




## 

## Step 6: Visualize results

In [11]:
from watertap.flowsheets.lsrro.multi_sweep import _lsrro_presweep as setup_sensitivity_sweep
from parameter_sweep import LinearSample, parameter_sweep
number_of_stages = 3
output_filename = f"param_sweep_output/{number_of_stages}_stage/results_LSRRO.csv"
sweep_params = {}
outputs = {}
nx = 3

# m = setup_sensitivity_sweep(number_of_stages=number_of_stages)
m.fs.feed.flow_mass_phase_comp.unfix()
m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"].fix()
m.fs.feed.properties[0].flow_vol_phase["Liq"].fix()
# Sweep parameters ------------------------------------------------------------------------

sweep_params["Feed Concentration"] = LinearSample(m.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"], 70, 150, nx)

sweep_params["Volumetric Recovery Rate"] = LinearSample(m.fs.water_recovery, 0.2, 0.65, nx)

# Outputs  -------------------------------------------------------------------------------
outputs["LCOW"] = m.fs.costing.LCOW
outputs["LCOW wrt Feed Flow"] = m.fs.costing.LCOW_feed
outputs["SEC"] = m.fs.costing.specific_energy_consumption
outputs["SEC wrt Feed"] = m.fs.costing.specific_energy_consumption_feed

sweep_sensitivity_results = parameter_sweep(
        m,
        sweep_params,
        outputs,
        csv_results_file_name=output_filename,
        optimize_function=lsrro.solve,
        interpolate_nan_outputs=True,
    )